In [2]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
prosumer = pd.read_csv('../data/raw/train.csv')
# Get net demand
prosumer['datetime'] = pd.to_datetime(prosumer['datetime'])
# prosumer['net_demand'] = prosumer[prosumer['is_consumption'] == 1].reset_index()['target'] - prosumer[prosumer['is_consumption'] == 0].reset_index()['target']
# net_prosumer = prosumer.drop(columns=['target', 'is_consumption','county','is_business','product_type','data_block_id','row_id','prediction_unit_id']).groupby('datetime').sum()
# Date selected due to corrupted data after
# net_prosumer = net_prosumer[net_prosumer.index < "2022-07-21 10:00:00"]
net_prosumer = prosumer[prosumer['is_consumption'] == 1].drop(columns='is_consumption')[['target', 'datetime']].groupby('datetime').sum() - prosumer[prosumer['is_consumption'] == 0].drop(columns='is_consumption')[['target', 'datetime']].groupby('datetime').sum()
net_prosumer['demand'] = prosumer[prosumer['is_consumption'] == 1].drop(columns='is_consumption')[['target', 'datetime']].groupby('datetime').sum()
net_prosumer['supply'] = prosumer[prosumer['is_consumption'] == 0].drop(columns='is_consumption')[['target', 'datetime']].groupby('datetime').sum()
net_prosumer.describe()

,target,demand,supply
count,15312.000000,15312.000000,15312.000000
mean,24491.749526,30356.185163,5864.435638
std,19095.117622,9943.956810,12483.735708
min,-81630.133000,0.000000,0.000000
25%,20476.536000,23726.007750,42.913750
50%,27624.398500,28921.808500,109.283500
75%,35263.906250,36038.074750,3888.608750
max,63251.063000,63354.178000,89930.586000


In [4]:
weather_history = pd.read_csv('../data/raw/historical_weather.csv')
weather_history['datetime'] = pd.to_datetime(weather_history['datetime'])
weather_history = weather_history.drop(columns=['latitude', 'longitude', 'data_block_id']).groupby("datetime").mean()
weather_history.describe()

,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation
count,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000,15275.000000
mean,5.740960,2.240304,0.049620,0.016049,1009.281510,60.912753,46.685981,34.407020,36.051410,4.849868,197.869471,106.490520,64.452931,42.037589
std,7.635356,6.846250,0.118073,0.051131,11.964149,31.213803,33.703470,29.682011,32.114876,1.849202,72.283113,171.397301,122.710267,58.382343
min,-14.283929,-17.221429,0.000000,0.000000,960.000000,0.000000,0.000000,0.000000,0.000000,1.091766,15.598214,0.000000,0.000000,0.000000
25%,-0.054464,-2.624554,0.000000,0.000000,1002.493750,33.589286,13.169643,7.000000,5.348214,3.397321,150.517857,0.901786,0.071429,0.678571
50%,5.192857,1.574107,0.004464,0.000000,1010.561607,67.428571,45.669643,27.553571,28.553571,4.662202,205.928571,10.625000,5.080357,3.830357
75%,10.893304,6.821429,0.038393,0.003750,1017.041071,89.924107,79.053571,57.125000,61.901786,6.162946,248.776786,146.035714,59.808036,75.491071
max,27.278571,19.208929,1.495536,0.898750,1043.284821,100.000000,100.000000,99.964286,100.000000,12.023313,356.303571,780.973214,670.464286,286.678571


In [5]:
weather_forecast = pd.read_csv('../data/raw/weather_forecast.csv')
weather_forecast['origin_datetime'] = pd.to_datetime(weather_forecast['origin_datetime'])
weather_forecast['forecast_datetime'] = weather_forecast['origin_datetime'] + pd.to_timedelta(weather_forecast['hours_ahead'], unit='h')
weather_forecast = weather_forecast.drop(columns=['latitude', 'longitude','origin_datetime','hours_ahead','data_block_id']).groupby("forecast_datetime").mean()
weather_forecast.columns = ['24h_forecasted_{}'.format(measurement) for measurement in weather_forecast.columns]
weather_forecast.index = weather_forecast.index.rename('datetime')
weather_forecast = weather_forecast.shift(-1, freq=pd.DateOffset(days=1))
weather_forecast.describe()

,24h_forecasted_temperature,24h_forecasted_dewpoint,24h_forecasted_cloudcover_high,24h_forecasted_cloudcover_low,24h_forecasted_cloudcover_mid,24h_forecasted_cloudcover_total,24h_forecasted_10_metre_u_wind_component,24h_forecasted_10_metre_v_wind_component,24h_forecasted_direct_solar_radiation,24h_forecasted_surface_solar_radiation_downwards,24h_forecasted_snowfall,24h_forecasted_total_precipitation
count,15312.000000,15312.000000,15312.000000,15312.000000,15312.000000,15312.000000,15312.000000,15312.000000,15312.000000,15312.000000,1.531200e+04,15312.000000
mean,5.754312,2.418110,0.394336,0.434182,0.358837,0.681559,1.260016,0.718833,151.461430,110.958922,2.531240e-05,0.000079
std,7.538573,6.862142,0.329696,0.349777,0.312534,0.304175,3.507764,3.694780,233.198492,182.204968,7.539590e-05,0.000143
min,-14.440721,-16.940665,0.000000,0.000000,0.000000,0.000000,-8.658795,-12.247292,-0.034623,-0.018016,-6.652304e-08,-0.000001
25%,-0.045048,-2.459937,0.082132,0.088906,0.076739,0.446990,-1.315232,-1.827729,0.000000,0.000000,0.000000e+00,0.000002
50%,5.108669,1.776591,0.333847,0.369020,0.280352,0.772140,1.611665,0.845021,1.133784,1.166048,0.000000e+00,0.000021
75%,10.861815,6.984979,0.669502,0.774975,0.601033,0.966972,3.829023,3.415557,255.221176,154.519287,1.044622e-05,0.000087
max,26.833043,18.842153,1.000000,1.000002,1.000000,1.000004,11.936635,11.132244,884.038571,798.967460,1.244646e-03,0.001538


In [6]:
gas_prices = pd.read_csv('../data/raw/gas_prices.csv')
# available before forecast date
gas_prices['forecast_date'] = pd.to_datetime(gas_prices['forecast_date'])
gas_prices = gas_prices.drop(columns = ['data_block_id', 'origin_date']).set_index('forecast_date')
gas_prices = gas_prices.resample('H').ffill()
gas_prices.index = gas_prices.index.rename('datetime')
gas_prices.describe()

,lowest_price_per_mwh,highest_price_per_mwh
count,15265.000000,15265.000000
mean,95.136249,107.865758
std,47.483048,54.668638
min,28.100000,34.000000
25%,60.000000,67.530000
50%,85.210000,93.470000
75%,109.000000,130.740000
max,250.000000,305.000000


In [7]:
electricity_prices = pd.read_csv('../data/raw/electricity_prices.csv')
# available before forecast date
electricity_prices['forecast_date'] = pd.to_datetime(electricity_prices['forecast_date'])
electricity_prices = electricity_prices.drop(columns = ['data_block_id', 'origin_date']).set_index('forecast_date')
electricity_prices.index = electricity_prices.index.rename('datetime')
electricity_prices.describe()

,euros_per_mwh
count,15286.000000
mean,157.064176
std,121.148625
min,-10.060000
25%,85.290000
50%,128.280000
75%,199.797500
max,4000.000000


In [8]:
client = pd.read_csv('../data/raw/client.csv')
client['date'] = pd.to_datetime(client['date'])
client = pd.get_dummies(client, columns=['product_type'])
client = client.drop(columns=['county','data_block_id']).groupby('date').sum()
client = client.resample('H').ffill()
client.index = client.index.rename('datetime')
client.describe()

,eic_count,installed_capacity,is_business,product_type_0,product_type_1,product_type_2,product_type_3
count,15241.000000,15241.000000,15241.000000,15241.000000,15241.000000,15241.000000,15241.0
mean,4830.246375,95541.519130,35.377994,5.566958,25.521291,4.823502,30.0
std,1231.323551,19717.032753,1.866956,0.793590,1.051787,0.869776,0.0
min,3156.000000,70651.260000,31.000000,4.000000,22.000000,2.000000,30.0
25%,3762.000000,76775.150000,35.000000,6.000000,26.000000,4.000000,30.0
50%,4620.000000,93411.910000,36.000000,6.000000,26.000000,5.000000,30.0
75%,5914.000000,111288.760000,37.000000,6.000000,26.000000,5.000000,30.0
max,7457.000000,148218.705000,38.000000,6.000000,26.000000,7.000000,30.0


In [9]:
final_df = net_prosumer.join(client, how='inner').join(weather_history, how='inner').join(gas_prices, how = 'inner').join(electricity_prices, how = 'inner').join(weather_forecast, how='left')

In [10]:
final_df = final_df[3:]
final_df['diff_demand'] = final_df['demand'].diff()
final_df['diff_supply'] = final_df['supply'].diff()
final_df = final_df[final_df['demand'] != 0]
final_df = final_df[final_df['supply'] != 0]

In [11]:
final_df = final_df.ffill()

In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15232 entries, 2021-09-01 03:00:00 to 2023-05-29 00:00:00
Data columns (total 41 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   target                                            15232 non-null  float64
 1   demand                                            15232 non-null  float64
 2   supply                                            15232 non-null  float64
 3   eic_count                                         15232 non-null  int64  
 4   installed_capacity                                15232 non-null  float64
 5   is_business                                       15232 non-null  int64  
 6   product_type_0                                    15232 non-null  int64  
 7   product_type_1                                    15232 non-null  int64  
 8   product_type_2                                    15232 non-n

In [17]:
final_df[int(len(final_df)*0.7):int(len(final_df)*0.7+len(final_df)*0.15)]

,target,demand,supply,eic_count,installed_capacity,is_business,product_type_0,product_type_1,product_type_2,product_type_3,...,24h_forecasted_cloudcover_mid,24h_forecasted_cloudcover_total,24h_forecasted_10_metre_u_wind_component,24h_forecasted_10_metre_v_wind_component,24h_forecasted_direct_solar_radiation,24h_forecasted_surface_solar_radiation_downwards,24h_forecasted_snowfall,24h_forecasted_total_precipitation,diff_demand,diff_supply
datetime,,,,,,,,,,,,,,,,,,,,,
2022-11-19 13:00:00,34887.373,35456.474,569.101,5673,108265.270,38,6,26,7,30,...,0.675926,0.868803,-2.909395,4.251086,85.858889,66.078160,0.000149,0.000149,176.998,-152.935
2022-11-19 14:00:00,36681.807,36812.634,130.827,5673,108265.270,38,6,26,7,30,...,0.601844,0.802115,-3.025622,4.572444,81.358832,40.108270,0.000140,0.000140,1356.160,-438.274
2022-11-19 15:00:00,36912.091,37024.188,112.097,5673,108265.270,38,6,26,7,30,...,0.531202,0.742555,-3.210620,4.842924,40.652381,8.818486,0.000128,0.000128,211.554,-18.730
2022-11-19 16:00:00,37948.883,38028.772,79.889,5673,108265.270,38,6,26,7,30,...,0.423116,0.673136,-3.331385,5.278376,0.040635,0.011119,0.000102,0.000102,1004.584,-32.208
2022-11-19 17:00:00,37573.403,37653.064,79.661,5673,108265.270,38,6,26,7,30,...,0.326072,0.574545,-3.303461,5.517465,0.000000,0.000000,0.000096,0.000096,-375.708,-0.228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-22 13:00:00,52099.103,58146.348,6047.245,6266,116496.635,35,6,26,4,30,...,0.999497,1.000000,2.186410,5.904500,33.685238,154.517171,0.000134,0.000136,932.826,7.736
2023-02-22 14:00:00,51885.304,56973.048,5087.744,6266,116496.635,35,6,26,4,30,...,0.999720,1.000000,2.169388,5.907989,12.575238,115.841835,0.000146,0.000150,-1173.300,-959.501
2023-02-22 15:00:00,52559.931,55521.222,2961.291,6266,116496.635,35,6,26,4,30,...,0.996990,1.000000,2.251959,5.924146,3.701587,73.809163,0.000135,0.000141,-1451.826,-2126.453


In [16]:
final_df.to_parquet('../data/processed/ENEFIT_dataset.parquet')